In [1]:
from model import load_model
from image_utils import imagenet_utils
import numpy as np
import os

from tensorflow import keras
from tqdm import tqdm
import cv2

In [2]:

folder='ILSVRC2012_img_val/'
test=load_model('VGG16')
model=test.model
input_dim=test.input_dim
layer_name='block5_conv3'


from explanation_models import gradcam,gradcam_robust,gradcam_plusplus,guided_backprop

g= gradcam(model,layer_name,input_dim)
dict={}

In [3]:
for image_name in tqdm(os.listdir(folder)):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input =keras.applications.vgg16.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap = -1*heatmap +1
        heatmap = np.clip(heatmap, 0., 1.)
        heatmap_list.append(heatmap)

    dict[image_name]=heatmap_list
